1) Prepare a classification model using SVM for salary data 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as pe
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['figure.dpi'] = 300
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
dftrain = pd.read_csv('SalaryData_Train.csv')
dftest = pd.read_csv('SalaryData_Test.csv')
dftrain.shape,dftest.shape
dftrain.head()
dftest.head()
# dftrain["Salary"].value_counts()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [3]:
# label encoding for salary class  <=50K    as 0 and Salary >50K  as 1
from sklearn.preprocessing import LabelEncoder
categ = ['workclass','education','maritalstatus','occupation','relationship','race','sex','native','Salary']

# Encode Categorical Columns
le = LabelEncoder()
dftrain[categ] = dftrain[categ].apply(le.fit_transform)
dftest[categ] = dftest[categ].apply(le.fit_transform)
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   age            30161 non-null  int64
 1   workclass      30161 non-null  int32
 2   education      30161 non-null  int32
 3   educationno    30161 non-null  int64
 4   maritalstatus  30161 non-null  int32
 5   occupation     30161 non-null  int32
 6   relationship   30161 non-null  int32
 7   race           30161 non-null  int32
 8   sex            30161 non-null  int32
 9   capitalgain    30161 non-null  int64
 10  capitalloss    30161 non-null  int64
 11  hoursperweek   30161 non-null  int64
 12  native         30161 non-null  int32
 13  Salary         30161 non-null  int32
dtypes: int32(9), int64(5)
memory usage: 2.2 MB


In [4]:
xtrain = dftrain.iloc[:,:-1]
ytrain = dftrain['Salary']

xtest = dftest.iloc[:,:-1]
ytest = dftest['Salary']

sc = StandardScaler()

xtrain = sc.fit_transform(xtrain)
pd.DataFrame(xtrain).describe()

sctest = StandardScaler()

xtest = sc.fit_transform(xtest)
pd.DataFrame(xtest).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12
count,1.506000e+04,1.506000e+04,1.506000e+04,1.506000e+04,1.506000e+04,1.506000e+04,1.506000e+04,1.506000e+04,1.506000e+04,1.506000e+04,1.506000e+04,1.506000e+04,1.506000e+04
mean,2.459373e-16,1.457577e-15,-8.307505e-17,7.519439e-18,-1.662474e-15,1.500202e-16,1.517600e-16,2.065782e-16,9.168850e-16,-1.014992e-15,-7.571043e-16,1.909200e-16,1.896845e-15
std,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00
min,-1.626902e+00,-2.291906e+00,-2.684800e+00,-3.561557e+00,-1.724114e+00,-1.489819e+00,-8.819898e-01,-4.451008e+00,-1.437128e+00,-1.454384e-01,-2.191694e-01,-3.312068e+00,-6.031159e+00
25%,-8.047938e-01,-2.223470e-01,-3.325499e-01,-4.348982e-01,-3.954633e-01,-9.923044e-01,-8.819898e-01,3.822302e-01,-1.437128e+00,-1.454384e-01,-2.191694e-01,-7.888904e-02,2.605807e-01
50%,-1.321597e-01,-2.223470e-01,1.901723e-01,-4.406595e-02,-3.954633e-01,2.725430e-03,-2.526239e-01,3.822302e-01,6.958322e-01,-1.454384e-01,-2.191694e-01,-7.888904e-02,2.605807e-01
75%,6.899487e-01,-2.223470e-01,4.515334e-01,1.128431e+00,9.331874e-01,7.489978e-01,1.006108e+00,3.822302e-01,6.958322e-01,-1.454384e-01,-2.191694e-01,3.356211e-01,2.605807e-01
max,3.828908e+00,3.916770e+00,1.235617e+00,2.300928e+00,2.261838e+00,1.744028e+00,2.264839e+00,3.822302e-01,6.958322e-01,1.283651e+01,9.060379e+00,4.812331e+00,6.006747e-01


In [5]:
svc = SVC(kernel='poly')
svc.fit(xtrain,ytrain)
ypred = svc.predict(xtest)

In [6]:
svc.score(xtrain,ytrain), svc.score(xtest,ytest)

(0.8432412718411193, 0.8406374501992032)

In [ ]:
params = {
    'kernel' : ['linear','poly','rbf'],
    'gamma' : [0.1,0.001,0.001,1,10],
    'C' : [0.1,0.001,1,10]
}

In [ ]:
grid = GridSearchCV(SVC(), param_grid=params, verbose=1)
grid.fit(xtrain,ytrain)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [ ]:
grid.best_params_

In [ ]:
svc = SVC(kernel='linear', C=10, gamma=0.1)
svc.fit(xtrain,ytrain)
ypred = svc.predict(xtest)

In [ ]:
svc.score(xtrain,ytrain), svc.score(xtest,ytest)